# LangChain 실습

In [29]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(len(OPENAI_API_KEY))

164


In [ ]:
# !pip install pydantic

In [ ]:
# !pip install openai langchain langchain-openai langchain-community

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 88.6 MB/s eta 0:00:00

  Attempting uninstall: requests

    Found existing installation: requests 2.32.3

    Uninstalling requests-2.32.3:

      Successfully uninstalled requests-2.32.3

   --------------- ------------------------ 3/8 [marshmallow]
  Attempting uninstall: pydantic-settings
   --------------- ------------------------ 3/8 [marshmallow]
    Found existing installation: pydantic-settings 2.6.1
   --------------- ------------------------ 3/8 [marshmallow]
    Uninstalling pydantic-settings-2.6.1:
   --------------- ------------------------ 3/8 [marshmallow]
      Successfully uninstalled pydantic-settings-2.6.1
   --------------- ------------------------ 3/8 [marshmallow]
   ----------------------------------- ---- 7/8 [langchain-community]
   ----------------------------------- ---- 7/8 [langchain-community]
   ----------------------------------- -

## 5-1. Data Connection

### Document loaders

In [ ]:
# !pip install GitPython

In [30]:
from langchain_classic.document_loaders import GitLoader

def file_filter(file_path):
    return file_path.endswith(".md")

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter
)
raw_docs = loader.load()
print(type(raw_docs), len(raw_docs))

<class 'list'> 36


### Document transformers

In [31]:
from langchain_classic.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(raw_docs)
print(type(docs), len(docs))
print(type(docs[0]), len(docs[0].page_content))

Created a chunk of size 1367, which is longer than the specified 1000
Created a chunk of size 1455, which is longer than the specified 1000
Created a chunk of size 1299, which is longer than the specified 1000


<class 'list'> 96
<class 'langchain_core.documents.base.Document'> 312


### Text embedding models

In [19]:
# !pip install tiktoken

In [32]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [33]:
query = "AWS의 S3에서 데이터를 읽기 위한 DocumentLoader가 있나요?"
vector = embeddings.embed_query(query)
print(type(vector), len(vector))
print(vector[:7])

<class 'list'> 1536
[-0.01556143257766962, -0.012867593206465244, 0.021737877279520035, -0.01614966429769993, -0.024117546156048775, 0.045186977833509445, -0.019705800339579582]


### Vector stores

In [23]:
!pip install chromadb

   ---------------------------------------- 0.0/21.4 MB ? eta -:--:--
   ---------------------------------------  21.2/21.4 MB 117.4 MB/s eta 0:00:01
   ---------------------------------------- 21.4/21.4 MB 93.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 94.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.5 MB ? eta -:--:--
   ------------- -------------------------- 4.5/13.5 MB 31.8 MB/s eta 0:00:01
   ---------------------------- ----------- 9.4/13.5 MB 31.0 MB/s eta 0:00:01
   ------------------------------ --------- 10.2/13.5 MB 18.1 MB/s eta 0:00:01
   ---------------------------------------- 13.5/13.5 MB 21.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 69.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/534.5 kB ? eta -:--:--
   ---------------------

In [34]:
from langchain_classic.vectorstores import Chroma
db = Chroma.from_documents(docs, embeddings)
print(type(db))

<class 'langchain_community.vectorstores.chroma.Chroma'>


### Retriever

In [35]:
retriever = db.as_retriever()
print(type(retriever))

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


In [38]:
query = "AWS S3에서 데이터를 불러올 수 있는 DocumentLoader가 있나요?"
context_docs = retriever.invoke(query)
print(type(context_docs), f"len = {len(context_docs)}")
first_doc = context_docs[0]
print(type(first_doc), f"metadata = {first_doc.metadata}")
print(type(first_doc.page_content), first_doc.page_content)

<class 'list'> len = 4
<class 'langchain_core.documents.base.Document'> metadata = {'file_path': '.devcontainer\\README.md', 'source': '.devcontainer\\README.md', 'file_type': '.md', 'file_name': 'README.md'}
<class 'str'> Then you will have a local cloned repo where you can contribute and then create pull requests.

If you already have VS Code and Docker installed, you can use the button above to get started. This will use VSCode to automatically install the Dev Containers extension if needed, clone the source code into a container volume, and spin up a dev container for use.

Alternatively you can also follow these steps to open this repo in a container using the VS Code Dev Containers extension:

1. If this is your first time using a development container, please ensure your system meets the pre-reqs (i.e. have Docker installed) in the [getting started steps](https://aka.ms/vscode-remote/containers/getting-started).

2. Open a locally cloned copy of the code:


### RetrievalQA（Chain）

In [28]:
from langchain_classic.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
qa_chain.invoke(query)

{'query': 'AWS S3에서 데이터를 불러올 수 있는 DocumentLoader가 있나요?',
 'result': '네, AWS S3에서 데이터를 불러올 수 있는 DocumentLoader가 있습니다.\n\n예를 들어, LangChain 라이브러리에서는 `S3FileLoader` 같은 클래스를 사용하여 S3 버킷에 저장된 파일을 불러올 수 있습니다. 이 로더는 AWS 자격 증명과 S3 버킷, 파일 경로를 지정하여 S3에 있는 문서를 로드할 수 있게 해줍니다.\n\n만약 LangChain에서 직접 제공하는 로더가 없다면, AWS SDK (예: boto3)를 사용해 S3에서 파일을 다운로드한 후, 해당 파일을 처리하는 DocumentLoader에 전달하는 방식으로 구현할 수도 있습니다.\n\n요약하면:\n- LangChain 등에서 `S3FileLoader` 같은 S3 전용 DocumentLoader가 존재할 수 있습니다.\n- 없다면 boto3로 S3에서 파일을 가져와서 일반 DocumentLoader에 넘기는 방법을 사용합니다.\n\n필요하시면 구체적인 코드 예제도 제공해 드릴 수 있습니다.'}